In [ ]:
import os
import threading
import datetime
import csv
import gspread
from google.oauth2.service_account import Credentials
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import serial
import serial.tools.list_ports
import queue
import time
import re
import webbrowser

# Column headers for Google Spreadsheet
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Google Sheets Scope
SCOPE = [
    "https://spreadsheets.google.com/feeds",
    'https://www.googleapis.com/auth/spreadsheets',
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

# Splash Screen Class
class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.attributes("-alpha", 1)

        # Resize splash screen for 7-inch screen (800x480 resolution)
        width, height = 800, 480
        self.root.geometry(f"{width}x{height}+0+0")
        self.root.configure(bg="black")

        self.label = tk.Label(
            self.root,
            text="McCutcheonLab Technologies",
            font=("Cascadia Code", 28, "bold"),
            bg="black",
            fg="violet"
        )
        self.label.pack(expand=True)
        self.fade_in_out(duration)

    def fade_in_out(self, duration):
        self.fade_in(1000, lambda: self.fade_out(2000, self.close))

    def fade_in(self, time_ms, callback):
        alpha = 0.0
        increment = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha < 1.0:
                alpha += increment
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def fade_out(self, time_ms, callback):
        alpha = 1.0
        decrement = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha > 0.0:
                alpha -= decrement
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def close(self):
        self.root.destroy()

# Main GUI Class
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("Realtime FED Monitor")
        self.root.geometry("800x480")  # Set for 7-inch touchscreen resolution

        # Variables for the GUI
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.json_path = tk.StringVar()
        self.spreadsheet_id = tk.StringVar()
        self.save_path = ""
        self.data_queue = queue.Queue()
        self.serial_ports = set(self.detect_serial_ports())
        self.threads = []
        self.port_widgets = {}
        self.port_queues = {}
        self.port_threads = {}
        self.identification_threads = {}
        self.identification_stop_events = {}
        self.log_queue = queue.Queue()
        self.recording_circle = None
        self.recording_label = None
        self.data_to_save = {}
        self.stop_event = threading.Event()
        self.logging_active = False
        self.data_saved = False  # Flag to prevent duplicate data saving

        self.gspread_client = None

        self.last_device_check_time = time.time()  # Initialize device check timer

        # Retry parameters
        self.retry_attempts = 5
        self.retry_delay = 2  # seconds

        self.setup_gui()

        # Schedule startup methods after mainloop starts with slight delays
        self.root.after(0, self.update_gui)
        self.root.after(100, self.show_instruction_popup)
        self.root.after(200, self.start_identification_threads)

        # Handle window close event
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def detect_serial_ports(self):
        ports = list(serial.tools.list_ports.comports())
        fed3_ports = []
        for port in ports:
            # Check for VID and PID matching your FED3 devices
            if port.vid == 0x239A and port.pid == 0x800B:
                fed3_ports.append(port.device)
        return fed3_ports

    def setup_gui(self):
        # Resize GUI elements to fit 7-inch screen
        self.root.grid_columnconfigure((0, 1, 2, 3), weight=1)
        self.root.grid_rowconfigure((0,1,2,3,4,5,6), weight=1)

        # Experimenter name input
        tk.Label(
            self.root,
            text="Your Name:",
            font=("Cascadia Code", 8, "bold")
        ).grid(column=0, row=0, sticky=tk.E, padx=2, pady=2)
        self.experimenter_entry = ttk.Entry(
            self.root,
            textvariable=self.experimenter_name,
            width=12
        )
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W, padx=2, pady=5)

        # Experiment name input
        tk.Label(
            self.root,
            text="Experiment Name:",
            font=("Cascadia Code", 8, "bold")
        ).grid(column=2, row=0, sticky=tk.E, padx=2, pady=2)
        self.experiment_entry = ttk.Entry(
            self.root,
            textvariable=self.experiment_name,
            width=15
        )
        self.experiment_entry.grid(column=3, row=0, sticky=tk.E, padx=2, pady=2)

        # JSON file path
        tk.Label(
            self.root,
            text="Google API JSON File:",
            font=("Cascadia Code", 8, "bold")
        ).grid(column=0, row=1, sticky=tk.E, padx=2, pady=2)
        self.json_entry = ttk.Entry(
            self.root,
            textvariable=self.json_path,
            width=25
        )
        self.json_entry.grid(column=1, row=1, columnspan=2, sticky=tk.W, padx=2, pady=2)
        self.browse_json_button = tk.Button(
            self.root,
            text="Browse",
            command=self.browse_json
        )
        self.browse_json_button.grid(column=3, row=1, padx=2, pady=2)

        # Google Spreadsheet ID
        tk.Label(
            self.root,
            text="Google Spreadsheet ID:",
            font=("Cascadia Code", 8, "bold")
        ).grid(column=0, row=2, sticky=tk.E, padx=2, pady=2)
        self.spreadsheet_entry = ttk.Entry(
            self.root,
            textvariable=self.spreadsheet_id,
            width=25
        )
        self.spreadsheet_entry.grid(column=1, row=2, columnspan=2, sticky=tk.W, padx=2, pady=2)

        # Start button
        self.start_button = tk.Button(
            self.root,
            text="START",
            font=("Cascadia Code", 10, "bold"),
            bg="green",
            fg="white",
            command=self.start_logging
        )
        self.start_button.grid(column=0, row=3, padx=5, pady=5, sticky=tk.W)

        # Stop button
        self.stop_button = tk.Button(
            self.root,
            text="STOP(SAVE & QUIT)",
            font=("Cascadia Code", 10, "bold"),
            bg="red",
            fg="white",
            command=self.stop_logging
        )
        self.stop_button.grid(column=1, row=3, padx=5, pady=5, sticky=tk.W)

        # Browse button for selecting data folder path
        self.browse_button = tk.Button(
            self.root,
            text="Browse Data Folder",
            font=("Cascadia Code", 10, "bold"),
            command=self.browse_folder,
            bg="gold",
            fg="blue"
        )
        self.browse_button.grid(column=2, row=3, padx=5, pady=5, sticky=tk.W)

        # Canvas for Recording Indicator (reduced size)
        self.canvas = tk.Canvas(self.root, width=100, height=100)
        self.canvas.grid(column=3, row=3, pady=5, sticky=tk.N)
        self.recording_circle = self.canvas.create_oval(25, 25, 75, 75, fill="Orange")
        self.recording_label = self.canvas.create_text(
            50, 90,
            text="Standby",
            font=("Cascadia Code", 12),
            fill="black"
        )

        # Port Status Frame
        self.ports_frame = tk.Frame(self.root)
        self.ports_frame.grid(column=0, row=4, columnspan=4, pady=10, sticky=(tk.N, tk.S, tk.E, tk.W))

        # Initialize ports
        if self.serial_ports:
            for idx, port in enumerate(self.serial_ports):
                self.initialize_port_widgets(port, idx)
        else:
            tk.Label(
                self.ports_frame,
                text="Connect your FED3 devices!",
                font=("Cascadia Code", 12),
                fg="red"
            ).pack()

        # Log Section
        log_frame = tk.Frame(self.root)
        log_frame.grid(column=0, row=5, columnspan=4, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))

        self.log_text = tk.Text(log_frame, height=5, width=100)
        self.log_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        log_scrollbar = ttk.Scrollbar(
            log_frame,
            orient="vertical",
            command=self.log_text.yview
        )
        log_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.log_text.configure(yscrollcommand=log_scrollbar.set)

        # Recording Indicator Frame (additional, if needed)
        # Can be used to display overall recording status

        # Bottom Frame for Copyright and Hyperlink
        bottom_frame = tk.Frame(self.root)
        bottom_frame.grid(column=0, row=6, columnspan=4, pady=5, sticky=(tk.S, tk.E, tk.W))

        tk.Label(
            bottom_frame,
            text="© 2024 McCutcheonLab | UiT | Norway",
            font=("Cascadia Code", 8),
            fg="royalblue"
        ).pack(pady=2)
        hyperlink_label = tk.Label(
            bottom_frame,
            text="Developed by Hamid Taghipourbibalan",
            font=("Cascadia Code", 8, "italic"),
            fg="blue",
            cursor="hand2"
        )
        hyperlink_label.pack(pady=2)
        hyperlink_label.bind("<Button-1>", lambda e: self.open_hyperlink("https://www.linkedin.com/in/hamid-taghipourbibalan-b7239088/"))

    def open_hyperlink(self, URL):
        webbrowser.open_new(URL)

    def show_instruction_popup(self):
        messagebox.showinfo("Instructions", "Make a right poke on FED3 to find the port!")

    def initialize_port_widgets(self, port, idx=None):
        if port in self.port_widgets:
            # Widgets already initialized
            return

        if idx is None:
            idx = len(self.port_widgets)

        port_name = os.path.basename(port)  # Use base name for display
        frame = ttk.LabelFrame(self.ports_frame, text=f"Port {port_name}")
        frame.grid(column=idx % 2, row=idx // 2, padx=5, pady=5, sticky=tk.W)
        status_label = ttk.Label(
            frame,
            text="Not Ready",
            font=("Cascadia Code", 8, "italic"),
            foreground="red"
        )
        status_label.grid(column=0, row=0, sticky=tk.W)
        text_widget = tk.Text(frame, width=35, height=5)
        text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))

        # Add the indicator canvas
        indicator_canvas = tk.Canvas(frame, width=20, height=20)
        indicator_canvas.grid(column=1, row=0, padx=5)
        indicator_circle = indicator_canvas.create_oval(5, 5, 15, 15, fill="gray")

        self.port_widgets[port] = {
            'status_label': status_label,
            'text_widget': text_widget,
            'indicator_canvas': indicator_canvas,
            'indicator_circle': indicator_circle
        }
        self.port_queues[port] = queue.Queue()  # Initialize queue for each port

        # Attempt to open serial port to check readiness
        try:
            ser = serial.Serial(port, 115200, timeout=1)
            ser.close()
            status_label.config(text="Ready", foreground="green")
        except serial.SerialException as e:
            status_label.config(text="Not Ready", foreground="red")
            self.log_queue.put(f"Error with port {port}: {e}")

    def browse_json(self):
        filename = filedialog.askopenfilename(title="Select JSON File", filetypes=[("JSON Files", "*.json")])
        if filename:
            self.json_path.set(filename)

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")
        if self.save_path:
            self.log_queue.put(f"Data folder selected: {self.save_path}")
            print(f"Save path selected: {self.save_path}")  # For debugging

    def start_identification_threads(self):
        # Start identification threads for each port
        for port in self.serial_ports:
            self.start_identification_thread(port)

    def start_identification_thread(self, port):
        if port in self.identification_threads:
            # Identification thread already running
            return

        stop_event = threading.Event()
        self.identification_stop_events[port] = stop_event

        def identification_with_delay():
            time.sleep(1)  # Wait for 1 second before attempting to connect
            self.identification_thread(port, stop_event)

        t = threading.Thread(target=identification_with_delay)
        t.daemon = True
        t.start()
        self.identification_threads[port] = t
        self.log_queue.put(f"Started identification thread for {port}.")

    def stop_identification_threads(self):
        # Stop identification threads
        for port, event in list(self.identification_stop_events.items()):
            event.set()
        for port, t in list(self.identification_threads.items()):
            t.join()
            self.log_queue.put(f"Stopped identification thread for {port}.")
            del self.identification_threads[port]
            del self.identification_stop_events[port]

    def identification_thread(self, port, stop_event):
        # Avoid opening the port if it's already in use
        if port in self.port_threads:
            self.log_queue.put(f"Skipping identification for {port} as it is already in use.")
            return

        try:
            ser = serial.Serial(port, 115200, timeout=0.1)
            # Get index of 'Event' column
            event_index_in_headers = column_headers.index("Event")
            event_index_in_data = event_index_in_headers - 1  # Adjust for timestamp

            while not stop_event.is_set():
                try:
                    data = ser.readline().decode('utf-8', errors='replace').strip()
                    if data:
                        data_list = data.split(",")
                        data_list = data_list[1:]  # Skip the first item if necessary
                        if len(data_list) == len(column_headers) - 1:
                            event_value = data_list[event_index_in_data].strip()
                            if event_value == "Right":
                                # Send message to GUI to trigger indicator
                                self.port_queues[port].put("RIGHT_POKE")
                except serial.SerialException as e:
                    self.log_queue.put(f"Device on {port} disconnected during identification: {e}")
                    break  # Exit the loop and end the thread
                except Exception as e:
                    self.log_queue.put(f"Error in identification thread for {port}: {e}")
        except serial.SerialException as e:
            # Log only if port is truly inaccessible
            if "PermissionError" not in str(e):  # Suppress permission-related errors
                self.log_queue.put(f"Could not open serial port {port} for identification: {e}")
        except Exception as e:
            self.log_queue.put(f"Unexpected error in identification thread for {port}: {e}")
        finally:
            try:
                ser.close()
            except:
                pass

    def start_logging(self):
        # Stop identification threads before starting logging
        self.stop_identification_threads()

        # Reset stop event and threads list
        self.stop_event.clear()
        self.threads = []
        self.logging_active = True

        # Normalize inputs
        self.experimenter_name.set(self.experimenter_name.get().strip().lower())
        self.experiment_name.set(self.experiment_name.get().strip().lower())
        self.json_path.set(self.json_path.get().strip())
        self.spreadsheet_id.set(self.spreadsheet_id.get().strip())

        # Validate input fields
        if not self.experimenter_name.get() or not self.experiment_name.get():
            messagebox.showerror("Error", "Please provide your name and experiment name.")
            return

        if not self.json_path.get() or not self.spreadsheet_id.get() or not self.save_path:
            messagebox.showerror("Error", "Please provide the JSON file path, Spreadsheet ID, and select a data folder.")
            return

        # Sanitize names to remove invalid characters
        experimenter_name = re.sub(r'[<>:"/\\|?*]', '_', self.experimenter_name.get())
        experiment_name = re.sub(r'[<>:"/\\|?*]', '_', self.experiment_name.get())
        self.experimenter_name.set(experimenter_name)
        self.experiment_name.set(experiment_name)

        # Google Sheets Client Setup with Retry Logic
        try:
            creds = Credentials.from_service_account_file(self.json_path.get(), scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            self.log_queue.put("Connected to Google Sheets!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to connect to Google Sheets: {e}")
            return

        # Disable input fields after pressing START
        self.disable_input_fields()

        # Change recording indicator to ON
        self.canvas.itemconfig(self.recording_circle, fill="yellow")
        self.canvas.itemconfig(self.recording_label, text="Logging...", fill="black")

        # Start serial threads with retry logic
        for port in list(self.serial_ports):
            self.start_logging_for_port(port)

    def disable_input_fields(self):
        # Disable the input fields
        self.experimenter_entry.config(state='disabled')
        self.experiment_entry.config(state='disabled')
        self.json_entry.config(state='disabled')
        self.spreadsheet_entry.config(state='disabled')
        self.browse_json_button.config(state='disabled')
        self.browse_button.config(state='disabled')
        self.start_button.config(state='disabled')

    def enable_input_fields(self):
        # Enable the input fields
        self.experimenter_entry.config(state='normal')
        self.experiment_entry.config(state='normal')
        self.json_entry.config(state='normal')
        self.spreadsheet_entry.config(state='normal')
        self.browse_json_button.config(state='normal')
        self.browse_button.config(state='normal')
        self.start_button.config(state='normal')

    def start_logging_for_port(self, port):
        if port in self.port_threads:
            # Already logging for this port
            return

        def attempt_connection(retries=self.retry_attempts, delay=self.retry_delay):
            for attempt in range(retries):
                try:
                    ser = serial.Serial(port, 115200, timeout=0.1)
                    self.data_to_save[port] = []
                    t = threading.Thread(
                        target=self.read_from_port,
                        args=(ser, f"Port_{os.path.basename(port)}", port)
                    )
                    t.daemon = True
                    t.start()
                    self.port_threads[port] = t
                    # Update status label if not already set to Ready
                    if self.port_widgets[port]['status_label'].cget("text") != "Ready":
                        self.port_widgets[port]['status_label'].config(text="Ready", foreground="green")
                    self.log_queue.put(f"Started logging from {port}.")
                    return
                except serial.SerialException as e:
                    self.log_queue.put(f"Attempt {attempt+1}: Error with port {port}: {e}")
                    time.sleep(delay)
            self.log_queue.put(f"Failed to connect to port {port} after {retries} attempts.")

        threading.Thread(target=attempt_connection).start()

    def stop_logging(self):
        # Signal threads to stop
        self.stop_event.set()
        self.logging_active = False
        self.log_queue.put("Stopping logging...")

        # Change recording indicator to OFF
        self.hide_recording_indicator()

        # Re-enable input fields
        self.enable_input_fields()

        # Start a background thread to join threads and save data
        threading.Thread(target=self._join_threads_and_save).start()

    def _join_threads_and_save(self):
        # Wait for logging threads to finish
        for port, t in list(self.port_threads.items()):
            t.join()
            self.log_queue.put(f"Logging thread for {port} has stopped.")
            del self.port_threads[port]

        # Save data
        self.save_all_data()
        self.data_saved = True  # Set flag to indicate data has been saved

        # Use root.after to call GUI methods from the main thread
        self.root.after(0, self._finalize_exit)

    def _finalize_exit(self):
        # Inform the user that data has been saved
        messagebox.showinfo("Data Saved", "All data has been saved locally.")

        # Destroy the GUI after the messagebox is closed
        self.root.destroy()

    def save_all_data(self):
        if not self.save_path:
            self.log_queue.put("Error: No save path specified.")
            return

        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_name = self.experimenter_name.get().lower().strip()
        experiment_name = self.experiment_name.get().lower().strip()

        experimenter_folder = os.path.join(self.save_path, experimenter_name)
        experiment_folder = os.path.join(experimenter_folder, f"{experiment_name}_{current_time}")
        os.makedirs(experiment_folder, exist_ok=True)

        for port, data_rows in self.data_to_save.items():
            if data_rows:  # Only save if there is data
                port_name = os.path.basename(port)
                safe_port_name = re.sub(r'[<>:"/\\|?*]', '_', port_name)
                filename_user = os.path.join(experiment_folder, f"{safe_port_name}_{current_time}.csv")
                try:
                    with open(filename_user, mode='w', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow(column_headers)
                        writer.writerows(data_rows)
                    self.log_queue.put(f"Data saved for {port} in {filename_user}.")
                except Exception as e:
                    self.log_queue.put(f"Failed to save data for {port}: {e}")

    def update_gui(self):
        # Update port text widgets
        for port_identifier, q in list(self.port_queues.items()):
            try:
                while True:
                    message = q.get_nowait()
                    if message == "RIGHT_POKE":
                        self.trigger_indicator(port_identifier)
                    else:
                        self.port_widgets[port_identifier]['text_widget'].insert(tk.END, message + "\n")
                        self.port_widgets[port_identifier]['text_widget'].see(tk.END)
            except queue.Empty:
                pass

        # Update log messages
        try:
            while True:
                log_message = self.log_queue.get_nowait()
                self.log_text.insert(tk.END, f"{datetime.datetime.now().strftime('%m/%d/%Y %H:%M:%S')}: {log_message}\n")
                self.log_text.see(tk.END)
        except queue.Empty:
            pass

        # Periodically check device connections every 5 seconds
        current_time = time.time()
        if current_time - self.last_device_check_time >= 5:
            self.check_device_connections()
            self.last_device_check_time = current_time

        # Schedule the next update
        self.root.after(100, self.update_gui)

    def check_device_connections(self):
        current_ports = set(self.detect_serial_ports())
        # Check for disconnected devices
        for port in list(self.serial_ports):
            if port not in current_ports:
                # Device disconnected
                self.serial_ports.remove(port)
                # Update status label to "Not Ready"
                if port in self.port_widgets:
                    self.port_widgets[port]['status_label'].config(text="Not Ready", foreground="red")
                self.log_queue.put(f"Device on {port} disconnected.")
                # Stop the thread associated with this port
                if port in self.port_threads:
                    # The thread will exit on its own due to exception handling
                    del self.port_threads[port]
                # Stop identification thread if running
                if port in self.identification_threads:
                    self.identification_stop_events[port].set()
                    self.identification_threads[port].join()
                    del self.identification_threads[port]
                    del self.identification_stop_events[port]

        # Check for newly connected devices
        for port in current_ports:
            if port not in self.serial_ports:
                # New device connected
                self.serial_ports.add(port)
                # Initialize widgets and queues for the new port
                idx = len(self.port_widgets)
                self.initialize_port_widgets(port, idx)
                # Start identification thread for the new device
                self.start_identification_thread(port)
                # Start logging from the new device if logging is active
                if self.logging_active:
                    self.start_logging_for_port(port)

    def read_from_port(self, ser, worksheet_name, port_identifier):
        try:
            spreadsheet = self.gspread_client.open_by_key(self.spreadsheet_id.get())
            sheet = self.get_or_create_worksheet(spreadsheet, worksheet_name)
            cached_data = []
            send_interval = 5  # seconds
            last_send_time = time.time()
            jam_event_occurred = False  # Flag to track JAM events during outage
            device_number = None  # Variable to store Device_Number

            # Get indices for 'Event' and 'Device_Number' columns
            event_index_in_headers = column_headers.index("Event")
            event_index_in_data = event_index_in_headers - 1  # Adjust for timestamp

            device_number_index_in_headers = column_headers.index("Device_Number")
            device_number_index_in_data = device_number_index_in_headers - 1  # Adjust for timestamp

            while not self.stop_event.is_set():
                try:
                    data = ser.readline().decode('utf-8', errors='replace').strip()
                except serial.SerialException as e:
                    # Device disconnected
                    self.log_queue.put(f"Device on {port_identifier} disconnected: {e}")
                    # Update status label to "Not Ready"
                    if port_identifier in self.port_widgets:
                        self.port_widgets[port_identifier]['status_label'].config(text="Not Ready", foreground="red")
                    break  # Exit the loop and end the thread

                if data:
                    data_list = data.split(",")
                    timestamp = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S.%f")[:-3]  # MM/DD/YYYY hh:mm:ss.SSS
                    data_list = data_list[1:]  # Skip the first item if necessary

                    if len(data_list) == len(column_headers) - 1:
                        event_value = data_list[event_index_in_data].strip()

                        if event_value == "JAM":
                            jam_event_occurred = True  # Set the flag if JAM occurs
                            # Do not append 'JAM' event to cached_data
                        else:
                            row_data = [timestamp] + data_list
                            cached_data.append(row_data)
                            # Instead of updating GUI directly, put message in queue
                            self.port_queues[port_identifier].put(f"Data logged: {data_list}")
                            self.data_to_save[port_identifier].append(row_data)

                            # Extract Device_Number
                            device_number = data_list[device_number_index_in_data].strip()

                            # Check for Right Poke in the 'Event' column
                            if event_value == "Right":
                                # Send message to GUI to trigger indicator
                                self.port_queues[port_identifier].put("RIGHT_POKE")
                    else:
                        self.log_queue.put(f"Warning: Data length mismatch on {port_identifier}")

                # Periodically attempt to send cached data and handle JAM event
                current_time = time.time()
                should_send = (current_time - last_send_time >= send_interval)
                if should_send:
                    if cached_data:
                        try:
                            # Append cached data to Google Sheets
                            sheet.append_rows(cached_data)
                            self.log_queue.put(f"Appended {len(cached_data)} rows from {port_identifier} to Google Sheets.")
                            cached_data.clear()
                        except Exception as e:
                            self.log_queue.put(f"Failed to send data to Google Sheets for {port_identifier}: {e}")

                    if jam_event_occurred:
                        try:
                            # Create a row with empty strings for all columns
                            jam_row = [''] * len(column_headers)
                            # Set the current timestamp in the first column
                            jam_row[0] = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S.%f")[:-3]
                            # Place 'JAM' in the 'Event' column
                            jam_row[event_index_in_headers] = "JAM"
                            # Include Device_Number
                            jam_row[device_number_index_in_headers] = device_number
                            # Append the JAM event to Google Sheets
                            sheet.append_row(jam_row)
                            # Log the action
                            self.log_queue.put(f"Additional JAM event logged for {port_identifier} due to prior outage.")
                            # Reset the JAM event flag
                            jam_event_occurred = False
                        except Exception as e:
                            self.log_queue.put(f"Failed to send JAM event to Google Sheets for {port_identifier}: {e}")

                    last_send_time = current_time

                # Sleep briefly to prevent CPU overuse
                time.sleep(0.1)
        except Exception as e:
            self.log_queue.put(f"Error reading from {ser.port}: {e}")
        finally:
            try:
                ser.close()
            except Exception:
                pass
            self.log_queue.put(f"Closed serial port {ser.port}")

    def get_or_create_worksheet(self, spreadsheet, title):
        try:
            return spreadsheet.worksheet(title)
        except gspread.exceptions.WorksheetNotFound:
            sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
            sheet.append_row(column_headers)
            return sheet

    def trigger_indicator(self, port_identifier):
        indicator_canvas = self.port_widgets[port_identifier]['indicator_canvas']
        indicator_circle = self.port_widgets[port_identifier]['indicator_circle']
        # Start blinking the indicator
        def blink(times):
            if times > 0:
                current_color = indicator_canvas.itemcget(indicator_circle, 'fill')
                next_color = 'red' if current_color == 'gray' else 'gray'
                indicator_canvas.itemconfig(indicator_circle, fill=next_color)
                self.root.after(250, lambda: blink(times -1))
            else:
                # Ensure the indicator is set back to 'gray' at the end
                indicator_canvas.itemconfig(indicator_circle, fill='gray')
        blink(6)  # Blink 6 times (3 cycles)

    def hide_recording_indicator(self):
        self.canvas.itemconfig(self.recording_circle, fill="red")
        self.canvas.itemconfig(self.recording_label, text="OFF", fill="red")

    def show_recording_indicator(self):
        self.canvas.itemconfig(self.recording_circle, fill="yellow")
        self.canvas.itemconfig(self.recording_label, text="Logging...", fill="black")

    def on_closing(self):
        # Check if data has already been saved
        if not self.data_saved:
            # Stop identification threads
            self.stop_identification_threads()

            # Signal logging threads to stop
            self.stop_event.set()
            self.logging_active = False

            # Wait for identification threads to finish
            for t in list(self.identification_threads.values()):
                t.join()

            # Wait for logging threads to finish
            for t in list(self.port_threads.values()):
                t.join()

            # Save data if any
            self.save_all_data()
            self.data_saved = True  # Set flag to indicate data has been saved

        # Destroy the GUI
        self.root.destroy()

# Main execution
if __name__ == "__main__":
    # Splash screen
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root, duration=7000)
    splash_root.after(7000, splash_screen.close)
    splash_root.mainloop()

    # Main GUI
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


# Code above seems to be working perfect, it recognizes ports and status of ports even after reconnecting FED3 and logs data correctly, I only need to adjust the gui window size

In [ ]:
import os
import threading
import datetime
import csv
import gspread
from google.oauth2.service_account import Credentials
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import serial
import serial.tools.list_ports
import queue
import time
import re
import webbrowser

# Column headers for Google Spreadsheet
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Google Sheets Scope
SCOPE = [
    "https://spreadsheets.google.com/feeds",
    'https://www.googleapis.com/auth/spreadsheets',
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.attributes("-alpha", 1)
        screen_width= self.root.winfo_screenwidth()
        screen_height= self.root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}+0+0")
        self.root.configure(bg="black")
        self.label = tk.Label(
            self.root,
            text="McCutcheonLab Technologies",
            font=("Cascadia Code", 32, "bold"),
            bg="black",
            fg="violet"
        )
        self.label.pack(expand=True)
        self.fade_in_out(duration)

    def fade_in_out(self, duration):
        self.fade_in(1000, lambda: self.fade_out(2000, self.close))

    def fade_in(self, time_ms, callback):
        alpha = 0.0
        increment = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha < 1.0:
                alpha += increment
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def fade_out(self, time_ms, callback):
        alpha = 1.0
        decrement = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha > 0.0:
                alpha -= decrement
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def close(self):
        self.root.destroy()

class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("Realtime FED Monitor")
        self.root.geometry("1200x800")

        # Variables
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.json_path = tk.StringVar()
        self.spreadsheet_id = tk.StringVar()
        self.save_path = ""
        self.serial_ports = set(self.detect_serial_ports())
        self.port_widgets = {}
        self.port_queues = {}
        self.port_threads = {}
        self.identification_threads = {}
        self.identification_stop_events = {}
        self.data_to_save = {}
        self.stop_event = threading.Event()
        self.logging_active = False
        self.data_saved = False
        self.gspread_client = None
        self.log_queue = queue.Queue()

        self.last_device_check_time = time.time()

        # Retry parameters
        self.retry_attempts = 5
        self.retry_delay = 2

        # Mappings to keep device continuity:
        # port_to_device_number: maps current port to known device_number (if identified)
        # device_number_to_widgets: maps device_number to the widget set (so that if device reconnects, we reuse same box)
        # device_number_to_port: current port that device_number is mapped to
        self.port_to_device_number = {}
        self.device_number_to_widgets = {}
        self.device_number_to_port = {}
        
        self.setup_gui()
        self.root.after(0, self.update_gui)
        self.root.after(100, self.show_instruction_popup)
        self.root.after(200, self.start_identification_threads)

        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def detect_serial_ports(self):
        ports = list(serial.tools.list_ports.comports())
        fed3_ports = []
        for port in ports:
            if port.vid == 0x239A and port.pid == 0x800B:
                fed3_ports.append(port.device)
        return fed3_ports

    def setup_gui(self):
        self.root.grid_columnconfigure((0, 1, 2, 3), weight=1)
        self.root.grid_rowconfigure((0,1,2,3,4,5,6), weight=1)

        tk.Label(self.root, text="Your Name:", font=("Cascadia Code", 12, "bold")).grid(column=0, row=0, sticky=tk.E, padx=5, pady=5)
        self.experimenter_entry = ttk.Entry(self.root, textvariable=self.experimenter_name, width=50)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W, padx=5, pady=5)

        tk.Label(self.root, text="Experiment Name:", font=("Cascadia Code", 12, "bold")).grid(column=2, row=0, sticky=tk.E, padx=5, pady=5)
        self.experiment_entry = ttk.Entry(self.root, textvariable=self.experiment_name, width=50)
        self.experiment_entry.grid(column=3, row=0, sticky=tk.W, padx=5, pady=5)

        tk.Label(self.root, text="Google API JSON File:", font=("Cascadia Code", 12, "bold")).grid(column=0, row=1, sticky=tk.E, padx=5, pady=5)
        self.json_entry = ttk.Entry(self.root, textvariable=self.json_path, width=50)
        self.json_entry.grid(column=1, row=1, sticky=tk.W, padx=5, pady=5)
        self.browse_json_button = tk.Button(self.root, text="Browse", command=self.browse_json, font=("Cascadia Code", 10))
        self.browse_json_button.grid(column=2, row=1, padx=5, pady=5, sticky=tk.W)

        tk.Label(self.root, text="Google Spreadsheet ID:", font=("Cascadia Code", 12, "bold")).grid(column=0, row=2, sticky=tk.E, padx=5, pady=5)
        self.spreadsheet_entry = ttk.Entry(self.root, textvariable=self.spreadsheet_id, width=50)
        self.spreadsheet_entry.grid(column=1, row=2, sticky=tk.W, padx=5, pady=5)

        self.start_button = tk.Button(self.root, text="START", font=("Cascadia Code", 12, "bold"), bg="green", fg="white", command=self.start_logging)
        self.start_button.grid(column=2, row=2, padx=10, pady=10, sticky=tk.W)

        self.stop_button = tk.Button(self.root, text="STOP(SAVE & QUIT)", font=("Cascadia Code", 12, "bold"), bg="red", fg="white", command=self.stop_logging)
        self.stop_button.grid(column=3, row=2, padx=10, pady=10, sticky=tk.W)

        self.browse_button = tk.Button(self.root, text="Browse Data Folder", font=("Cascadia Code", 12, "bold"), command=self.browse_folder, bg="gold", fg="blue")
        self.browse_button.grid(column=0, row=3, padx=10, pady=10, sticky=tk.W)

        # Indicator in the middle
        indicator_frame = tk.Frame(self.root)
        indicator_frame.grid(column=1, row=3, columnspan=2, pady=10)
        self.canvas = tk.Canvas(indicator_frame, width=100, height=100)
        self.canvas.pack()
        self.recording_circle = self.canvas.create_oval(25, 25, 75, 75, fill="Orange")
        self.recording_label = self.canvas.create_text(50, 90, text="Standby", font=("Cascadia Code", 12), fill="black")

        # Ports frame with scrollbar
        ports_frame_container = tk.Frame(self.root)
        ports_frame_container.grid(column=0, row=4, columnspan=4, pady=20, sticky=(tk.N, tk.S, tk.E, tk.W))
        ports_canvas = tk.Canvas(ports_frame_container)
        ports_scrollbar = ttk.Scrollbar(ports_frame_container, orient="vertical", command=ports_canvas.yview)
        ports_canvas.configure(yscrollcommand=ports_scrollbar.set)
        ports_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        ports_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.ports_frame = tk.Frame(ports_canvas)
        ports_canvas.create_window((0,0), window=self.ports_frame, anchor="nw")
        self.ports_frame.bind("<Configure>", lambda event: ports_canvas.configure(scrollregion=ports_canvas.bbox("all")))

        # Initialize ports
        if self.serial_ports:
            for idx, port in enumerate(self.serial_ports):
                # Create a temporary widget box for this port
                self.create_port_widget_for_port(port, idx)
                self.try_open_port_for_status(port)
        else:
            tk.Label(self.ports_frame, text="Connect your FED3 devices!", font=("Cascadia Code", 14), fg="red").pack()

        # Log Section
        log_frame = tk.Frame(self.root)
        log_frame.grid(column=0, row=5, columnspan=4, pady=10, sticky=(tk.N, tk.S, tk.E, tk.W))

        self.log_text = tk.Text(log_frame, height=10, width=130, font=("Cascadia Code", 10))
        self.log_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        log_scrollbar = ttk.Scrollbar(log_frame, orient="vertical", command=self.log_text.yview)
        log_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.log_text.configure(yscrollcommand=log_scrollbar.set)

        # Bottom Frame
        bottom_frame = tk.Frame(self.root)
        bottom_frame.grid(column=0, row=6, columnspan=4, pady=10, sticky=(tk.S, tk.E, tk.W))
        tk.Label(bottom_frame, text="© 2024 McCutcheonLab | UiT | Norway", font=("Cascadia Code", 10), fg="royalblue").pack(pady=5)
        hyperlink_label = tk.Label(bottom_frame, text="Developed by Hamid Taghipourbibalan", font=("Cascadia Code", 10, "italic"), fg="blue", cursor="hand2")
        hyperlink_label.pack(pady=5)
        hyperlink_label.bind("<Button-1>", lambda e: self.open_hyperlink("https://www.linkedin.com/in/hamid-taghipourbibalan-b7239088/"))

    def open_hyperlink(self, URL):
        webbrowser.open_new(URL)

    def show_instruction_popup(self):
        messagebox.showinfo("Instructions", "Make a right poke on FED3 to find the port!")

    def create_port_widget_for_port(self, port, idx=None):
        # Create a widget set for the given port (temporary)
        # If device_number is later identified and already known, we will re-map
        if idx is None:
            idx = len(self.port_widgets)
        port_name = os.path.basename(port)
        frame = ttk.LabelFrame(self.ports_frame, text=f"Port {port_name}")
        frame.grid(column=idx % 2, row=idx // 2, padx=10, pady=10, sticky=tk.W)
        status_label = ttk.Label(frame, text="Not Ready", font=("Cascadia Code", 10, "italic"), foreground="red")
        status_label.grid(column=0, row=0, sticky=tk.W)
        text_widget = tk.Text(frame, width=40, height=6, font=("Cascadia Code", 9))
        text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))
        indicator_canvas = tk.Canvas(frame, width=20, height=20)
        indicator_canvas.grid(column=1, row=0, padx=5)
        indicator_circle = indicator_canvas.create_oval(5, 5, 15, 15, fill="gray")

        self.port_widgets[port] = {
            'frame': frame,
            'status_label': status_label,
            'text_widget': text_widget,
            'indicator_canvas': indicator_canvas,
            'indicator_circle': indicator_circle
        }
        self.port_queues[port] = queue.Queue()

    def try_open_port_for_status(self, port):
        try:
            ser = serial.Serial(port, 115200, timeout=1)
            ser.close()
            self.port_widgets[port]['status_label'].config(text="Ready", foreground="green")
        except serial.SerialException:
            self.port_widgets[port]['status_label'].config(text="Not Ready", foreground="red")

    def browse_json(self):
        filename = filedialog.askopenfilename(title="Select JSON File", filetypes=[("JSON Files", "*.json")])
        if filename:
            self.json_path.set(filename)

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")
        if self.save_path:
            self.log_queue.put(f"Data folder selected: {self.save_path}")

    def start_identification_threads(self):
        # Start identification for devices that don't have a known device_number
        for port in list(self.serial_ports):
            if port not in self.port_to_device_number:
                self.start_identification_thread(port)

    def start_identification_thread(self, port):
        if port in self.identification_threads:
            return
        stop_event = threading.Event()
        self.identification_stop_events[port] = stop_event

        def identification_with_delay():
            time.sleep(1)
            self.identification_thread(port, stop_event)

        t = threading.Thread(target=identification_with_delay)
        t.daemon = True
        t.start()
        self.identification_threads[port] = t
        self.log_queue.put(f"Started identification thread for {port}.")

    def stop_identification_threads(self):
        for port, event in list(self.identification_stop_events.items()):
            event.set()
        for port, t in list(self.identification_threads.items()):
            t.join()
            self.log_queue.put(f"Stopped identification thread for {port}.")
            del self.identification_threads[port]
            del self.identification_stop_events[port]

    def identification_thread(self, port, stop_event):
        if port in self.port_threads:
            return
        try:
            ser = serial.Serial(port, 115200, timeout=0.1)
            event_index = column_headers.index("Event") - 1
            device_number_index = column_headers.index("Device_Number") - 1
            device_number_found = None
            while not stop_event.is_set():
                try:
                    data = ser.readline().decode('utf-8', errors='replace').strip()
                    if data:
                        data_list = data.split(",")[1:]
                        if len(data_list) == len(column_headers)-1:
                            # Check device_number
                            dn = data_list[device_number_index].strip()
                            if dn:
                                device_number_found = dn
                                break
                except serial.SerialException:
                    break
                except:
                    # Ignore errors and continue reading
                    pass
            if device_number_found:
                self.log_queue.put(f"Identified device_number={device_number_found} on port={port}")
                self.register_device_number(port, device_number_found)
        except serial.SerialException:
            pass
        finally:
            try:
                ser.close()
            except:
                pass

    def register_device_number(self, port, device_number):
        # If we have seen this device_number before, reuse its widget
        if device_number in self.device_number_to_widgets:
            # We already have a widget set for this device_number from a previous run or connection
            old_widgets = self.device_number_to_widgets[device_number]
            # Remove the temporary widget associated to this port if different
            if port in self.port_widgets and self.port_widgets[port] is not old_widgets:
                # Destroy the newly created frame for this port
                new_widgets = self.port_widgets[port]
                new_widgets['frame'].destroy()
                del self.port_widgets[port]
                del self.port_queues[port]

            # Map this port to the existing device_number
            self.port_to_device_number[port] = device_number
            self.device_number_to_port[device_number] = port

        else:
            # First time seeing this device_number
            self.port_to_device_number[port] = device_number
            self.device_number_to_port[device_number] = port
            # Associate this widget to the device_number
            self.device_number_to_widgets[device_number] = self.port_widgets[port]

        # Start logging if logging is active
        if self.logging_active:
            self.start_logging_for_port(port)

    def start_logging(self):
        self.stop_identification_threads()
        self.stop_event.clear()
        self.logging_active = True

        # Normalize inputs
        self.experimenter_name.set(self.experimenter_name.get().strip().lower())
        self.experiment_name.set(self.experiment_name.get().strip().lower())
        self.json_path.set(self.json_path.get().strip())
        self.spreadsheet_id.set(self.spreadsheet_id.get().strip())

        if not self.experimenter_name.get() or not self.experiment_name.get():
            messagebox.showerror("Error", "Please provide your name and experiment name.")
            return
        if not self.json_path.get() or not self.spreadsheet_id.get() or not self.save_path:
            messagebox.showerror("Error", "Please provide the JSON file, Spreadsheet ID, and data folder.")
            return

        try:
            creds = Credentials.from_service_account_file(self.json_path.get(), scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            self.log_queue.put("Connected to Google Sheets!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to connect to Google Sheets: {e}")
            return

        self.disable_input_fields()
        self.canvas.itemconfig(self.recording_circle, fill="yellow")
        self.canvas.itemconfig(self.recording_label, text="Logging...", fill="black")

        # Start logging for identified devices
        for port in list(self.serial_ports):
            if port in self.port_to_device_number:
                self.start_logging_for_port(port)
            # If not identified yet, logging will start once identification is done

    def disable_input_fields(self):
        self.experimenter_entry.config(state='disabled')
        self.experiment_entry.config(state='disabled')
        self.json_entry.config(state='disabled')
        self.spreadsheet_entry.config(state='disabled')
        self.browse_json_button.config(state='disabled')
        self.browse_button.config(state='disabled')
        self.start_button.config(state='disabled')

    def enable_input_fields(self):
        self.experimenter_entry.config(state='normal')
        self.experiment_entry.config(state='normal')
        self.json_entry.config(state='normal')
        self.spreadsheet_entry.config(state='normal')
        self.browse_json_button.config(state='normal')
        self.browse_button.config(state='normal')
        self.start_button.config(state='normal')

    def start_logging_for_port(self, port):
        if port in self.port_threads or not self.logging_active:
            return

        def attempt_connection(retries=self.retry_attempts, delay=self.retry_delay):
            for attempt in range(retries):
                try:
                    ser = serial.Serial(port, 115200, timeout=0.1)
                    self.data_to_save[port] = []
                    device_number = self.port_to_device_number.get(port)
                    # Worksheet name based on device_number
                    if device_number:
                        worksheet_name = f"Device_{device_number}"
                    else:
                        worksheet_name = f"Port_{os.path.basename(port)}"

                    t = threading.Thread(target=self.read_from_port, args=(ser, port, worksheet_name))
                    t.daemon = True
                    t.start()
                    self.port_threads[port] = t
                    self.port_widgets[port]['status_label'].config(text="Ready", foreground="green")
                    self.log_queue.put(f"Started logging from {port}.")
                    return
                except serial.SerialException as e:
                    self.log_queue.put(f"Attempt {attempt+1}: Error with port {port}: {e}")
                    time.sleep(delay)
            self.log_queue.put(f"Failed to connect to port {port} after {retries} attempts.")

        threading.Thread(target=attempt_connection).start()

    def stop_logging(self):
        self.stop_event.set()
        self.logging_active = False
        self.log_queue.put("Stopping logging...")
        self.hide_recording_indicator()
        self.enable_input_fields()
        threading.Thread(target=self._join_threads_and_save).start()

    def _join_threads_and_save(self):
        for port, t in list(self.port_threads.items()):
            t.join()
            self.log_queue.put(f"Logging thread for {port} has stopped.")
            del self.port_threads[port]

        self.save_all_data()
        self.data_saved = True
        self.root.after(0, self._finalize_exit)

    def _finalize_exit(self):
        messagebox.showinfo("Data Saved", "All data has been saved locally.")
        self.root.destroy()

    def save_all_data(self):
        if not self.save_path:
            self.log_queue.put("Error: No save path specified.")
            return

        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_name = self.experimenter_name.get().lower().strip()
        experiment_name = self.experiment_name.get().lower().strip()

        experimenter_folder = os.path.join(self.save_path, experimenter_name)
        experiment_folder = os.path.join(experimenter_folder, f"{experiment_name}_{current_time}")
        os.makedirs(experiment_folder, exist_ok=True)

        for port, data_rows in self.data_to_save.items():
            if data_rows:
                port_name = os.path.basename(port)
                safe_port_name = re.sub(r'[<>:"/\\|?*]', '_', port_name)
                filename_user = os.path.join(experiment_folder, f"{safe_port_name}_{current_time}.csv")
                try:
                    with open(filename_user, mode='w', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow(column_headers)
                        writer.writerows(data_rows)
                    self.log_queue.put(f"Data saved for {port} in {filename_user}.")
                except Exception as e:
                    self.log_queue.put(f"Failed to save data for {port}: {e}")

    def update_gui(self):
        # Update port text widgets and blink indicator on RIGHT_POKE always
        for port_identifier, q in list(self.port_queues.items()):
            try:
                while True:
                    message = q.get_nowait()
                    if message == "RIGHT_POKE":
                        # Blink even if not started
                        self.trigger_indicator(port_identifier)
                    else:
                        self.port_widgets[port_identifier]['text_widget'].insert(tk.END, message + "\n")
                        self.port_widgets[port_identifier]['text_widget'].see(tk.END)
            except queue.Empty:
                pass

        # Update log messages
        try:
            while True:
                log_message = self.log_queue.get_nowait()
                self.log_text.insert(tk.END, f"{datetime.datetime.now().strftime('%m/%d/%Y %H:%M:%S')}: {log_message}\n")
                self.log_text.see(tk.END)
        except queue.Empty:
            pass

        current_time = time.time()
        if current_time - self.last_device_check_time >= 5:
            self.check_device_connections()
            self.last_device_check_time = current_time

        self.root.after(100, self.update_gui)

    def check_device_connections(self):
        current_ports = set(self.detect_serial_ports())
        # Disconnected devices
        for port in list(self.serial_ports):
            if port not in current_ports:
                self.serial_ports.remove(port)
                if port in self.port_widgets:
                    self.port_widgets[port]['status_label'].config(text="Not Ready", foreground="red")
                self.log_queue.put(f"Device on {port} disconnected.")
                if port in self.port_threads:
                    del self.port_threads[port]
                if port in self.identification_threads:
                    self.identification_stop_events[port].set()
                    self.identification_threads[port].join()
                    del self.identification_threads[port]
                    del self.identification_stop_events[port]
                # Do not delete device_number_to_widgets here; if device reconnects, we reuse

        # New devices
        for port in current_ports:
            if port not in self.serial_ports:
                self.serial_ports.add(port)
                # Check if we already know a device_number from before by quickly trying identification
                # Actually we must create a port widget if no known device_number widgets:
                if port not in self.port_widgets:
                    idx = len(self.port_widgets)
                    self.create_port_widget_for_port(port, idx)
                self.try_open_port_for_status(port)

                # If device_number known from previous run?
                # We can't know device_number until identified. Start identification if not known.
                if port not in self.port_to_device_number:
                    self.start_identification_thread(port)
                else:
                    # device_number known, if logging active start logging
                    if self.logging_active:
                        self.start_logging_for_port(port)

    def read_from_port(self, ser, port_identifier, worksheet_name):
        cached_data = []
        send_interval = 5
        last_send_time = time.time()
        jam_event_occurred = False
        device_number = self.port_to_device_number.get(port_identifier, None)

        event_index_in_headers = column_headers.index("Event")
        event_index_in_data = event_index_in_headers - 1
        device_number_index_in_headers = column_headers.index("Device_Number")
        device_number_index_in_data = device_number_index_in_headers - 1

        sheet = None

        while not self.stop_event.is_set():
            try:
                data = ser.readline().decode('utf-8', errors='replace').strip()
            except serial.SerialException as e:
                self.log_queue.put(f"Device on {port_identifier} disconnected: {e}")
                if port_identifier in self.port_widgets:
                    self.port_widgets[port_identifier]['status_label'].config(text="Not Ready", foreground="red")
                break

            if data:
                data_list = data.split(",")[1:]
                timestamp = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S.%f")[:-3]
                if len(data_list) == len(column_headers) - 1:
                    event_value = data_list[event_index_in_data].strip()
                    current_device_number = data_list[device_number_index_in_data].strip()

                    if device_number is None and current_device_number:
                        device_number = current_device_number
                        self.port_to_device_number[port_identifier] = device_number
                        if device_number not in self.device_number_to_widgets:
                            self.device_number_to_widgets[device_number] = self.port_widgets[port_identifier]
                        self.device_number_to_port[device_number] = port_identifier
                        worksheet_name = f"Device_{device_number}"

                    row_data = [timestamp] + data_list
                    cached_data.append(row_data)
                    self.port_queues[port_identifier].put(f"Data logged: {data_list}")
                    self.data_to_save.setdefault(port_identifier, []).append(row_data)

                    if event_value == "JAM":
                        jam_event_occurred = True
                    if event_value == "Right":
                        self.port_queues[port_identifier].put("RIGHT_POKE")
                else:
                    self.log_queue.put(f"Warning: Data length mismatch on {port_identifier}")

            # Attempt sending data if internet is available and logging active
            current_time = time.time()
            should_send = (current_time - last_send_time >= send_interval)
            if should_send and device_number and self.gspread_client and self.spreadsheet_id.get():
                # Try to get/create sheet if not available yet
                if sheet is None:
                    try:
                        spreadsheet = self.gspread_client.open_by_key(self.spreadsheet_id.get())
                        sheet = self.get_or_create_worksheet(spreadsheet, worksheet_name)
                    except Exception as e:
                        self.log_queue.put(f"Failed to access sheet for {port_identifier}: {e}")

                if sheet and cached_data:
                    # Try sending cached data
                    try:
                        sheet.append_rows(cached_data)
                        self.log_queue.put(f"Appended {len(cached_data)} rows from {port_identifier} to Google Sheets.")
                        cached_data.clear()
                    except Exception as e:
                        self.log_queue.put(f"Failed to send data to Google Sheets for {port_identifier}: {e}")

                if jam_event_occurred and sheet:
                    try:
                        jam_row = [''] * len(column_headers)
                        jam_row[0] = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S.%f")[:-3]
                        jam_row[event_index_in_headers] = "JAM"
                        if device_number:
                            jam_row[device_number_index_in_headers] = device_number
                        sheet.append_row(jam_row)
                        self.log_queue.put(f"JAM event logged for {port_identifier}")
                        jam_event_occurred = False
                    except Exception as e:
                        self.log_queue.put(f"Failed to send JAM event to Google Sheets for {port_identifier}: {e}")

                last_send_time = current_time

            time.sleep(0.1)

        try:
            ser.close()
        except:
            pass
        self.log_queue.put(f"Closed serial port {ser.port}")

    def get_or_create_worksheet(self, spreadsheet, title):
        try:
            return spreadsheet.worksheet(title)
        except gspread.exceptions.WorksheetNotFound:
            sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
            sheet.append_row(column_headers)
            return sheet

    def trigger_indicator(self, port_identifier):
        indicator_canvas = self.port_widgets[port_identifier]['indicator_canvas']
        indicator_circle = self.port_widgets[port_identifier]['indicator_circle']
        def blink(times):
            if times > 0:
                current_color = indicator_canvas.itemcget(indicator_circle, 'fill')
                next_color = 'red' if current_color == 'gray' else 'gray'
                indicator_canvas.itemconfig(indicator_circle, fill=next_color)
                self.root.after(250, lambda: blink(times - 1))
            else:
                indicator_canvas.itemconfig(indicator_circle, fill='gray')
        blink(6)

    def hide_recording_indicator(self):
        self.canvas.itemconfig(self.recording_circle, fill="red")
        self.canvas.itemconfig(self.recording_label, text="OFF", fill="red")

    def on_closing(self):
        if not self.data_saved:
            self.stop_identification_threads()
            self.stop_event.set()
            self.logging_active = False

            for t in list(self.identification_threads.values()):
                t.join()

            for t in list(self.port_threads.values()):
                t.join()

            self.save_all_data()
            self.data_saved = True

        self.root.destroy()


if __name__ == "__main__":
    # Splash screen
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root, duration=7000)
    splash_root.after(7000, splash_screen.close)
    splash_root.mainloop()

    # Main GUI
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


# below working on internet outage handling

In [ ]:
import os
import threading
import datetime
import csv
import gspread
from google.oauth2.service_account import Credentials
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import serial
import serial.tools.list_ports
import queue
import time
import re
import webbrowser

# Column headers for Google Spreadsheet
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Google Sheets Scope
SCOPE = [
    "https://spreadsheets.google.com/feeds",
    'https://www.googleapis.com/auth/spreadsheets',
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.attributes("-alpha", 1)
        screen_width= self.root.winfo_screenwidth()
        screen_height= self.root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}+0+0")
        self.root.configure(bg="black")

        self.label = tk.Label(
            self.root,
            text="McCutcheonLab Technologies",
            font=("Cascadia Code", 32, "bold"),
            bg="black",
            fg="violet"
        )
        self.label.pack(expand=True)
        self.fade_in_out(duration)

    def fade_in_out(self, duration):
        self.fade_in(1000, lambda: self.fade_out(2000, self.close))

    def fade_in(self, time_ms, callback):
        alpha = 0.0
        increment = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha < 1.0:
                alpha += increment
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def fade_out(self, time_ms, callback):
        alpha = 1.0
        decrement = 1 / (time_ms // 50)
        def fade():
            nonlocal alpha
            if alpha > 0.0:
                alpha -= decrement
                self.root.attributes("-alpha", alpha)
                self.root.after(50, fade)
            else:
                callback()
        fade()

    def close(self):
        self.root.destroy()

class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("Realtime FED Monitor")
        self.root.geometry("1200x800")

        # Variables
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.json_path = tk.StringVar()
        self.spreadsheet_id = tk.StringVar()
        self.save_path = ""
        self.serial_ports = set(self.detect_serial_ports())
        self.port_widgets = {}           # port -> widget dict
        self.port_queues = {}            # port -> queue
        self.port_threads = {}           # port -> thread for logging
        self.identification_threads = {} # port -> thread for identification
        self.identification_stop_events = {}
        self.data_to_save = {}           # port -> data rows
        self.stop_event = threading.Event()
        self.logging_active = False
        self.data_saved = False
        self.gspread_client = None
        self.log_queue = queue.Queue()

        self.last_device_check_time = time.time()

        # Retry parameters
        self.retry_attempts = 5
        self.retry_delay = 2

        # Mappings to maintain device continuity
        # device_number known -> reuse old widget if exists
        self.port_to_device_number = {}
        self.device_number_to_widgets = {}  # device_number -> widget sets
        self.device_number_to_port = {}      # device_number -> current port

        # Keep track of newly connected ports that are not yet identified
        # We don't create a widget until identification finishes and we know if device_number is known or not.
        self.unidentified_ports = set(self.serial_ports)

        self.setup_gui()
        self.root.after(0, self.update_gui)
        self.root.after(100, self.show_instruction_popup)
        self.root.after(200, self.start_identification_threads)

        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def detect_serial_ports(self):
        ports = list(serial.tools.list_ports.comports())
        fed3_ports = []
        for port in ports:
            if port.vid == 0x239A and port.pid == 0x800B:
                fed3_ports.append(port.device)
        return fed3_ports

    def setup_gui(self):
        self.root.grid_columnconfigure((0, 1, 2, 3), weight=1)
        self.root.grid_rowconfigure((0,1,2,3,4,5,6), weight=1)

        tk.Label(self.root, text="Your Name:", font=("Cascadia Code", 12, "bold")).grid(column=0, row=0, sticky=tk.E, padx=5, pady=5)
        self.experimenter_entry = ttk.Entry(self.root, textvariable=self.experimenter_name, width=50)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W, padx=5, pady=5)

        tk.Label(self.root, text="Experiment Name:", font=("Cascadia Code", 12, "bold")).grid(column=2, row=0, sticky=tk.E, padx=5, pady=5)
        self.experiment_entry = ttk.Entry(self.root, textvariable=self.experiment_name, width=50)
        self.experiment_entry.grid(column=3, row=0, sticky=tk.W, padx=5, pady=5)

        tk.Label(self.root, text="Google API JSON File:", font=("Cascadia Code", 12, "bold")).grid(column=0, row=1, sticky=tk.E, padx=5, pady=5)
        self.json_entry = ttk.Entry(self.root, textvariable=self.json_path, width=50)
        self.json_entry.grid(column=1, row=1, sticky=tk.W, padx=5, pady=5)
        self.browse_json_button = tk.Button(self.root, text="Browse", command=self.browse_json, font=("Cascadia Code", 10))
        self.browse_json_button.grid(column=2, row=1, padx=5, pady=5, sticky=tk.W)

        tk.Label(self.root, text="Google Spreadsheet ID:", font=("Cascadia Code", 12, "bold")).grid(column=0, row=2, sticky=tk.E, padx=5, pady=5)
        self.spreadsheet_entry = ttk.Entry(self.root, textvariable=self.spreadsheet_id, width=50)
        self.spreadsheet_entry.grid(column=1, row=2, sticky=tk.W, padx=5, pady=5)

        self.start_button = tk.Button(self.root, text="START", font=("Cascadia Code", 12, "bold"), bg="green", fg="white", command=self.start_logging)
        self.start_button.grid(column=2, row=2, padx=10, pady=10, sticky=tk.W)
        self.stop_button = tk.Button(self.root, text="STOP(SAVE & QUIT)", font=("Cascadia Code", 12, "bold"), bg="red", fg="white", command=self.stop_logging)
        self.stop_button.grid(column=3, row=2, padx=10, pady=10, sticky=tk.W)
        self.browse_button = tk.Button(self.root, text="Browse Data Folder", font=("Cascadia Code", 12, "bold"), command=self.browse_folder, bg="gold", fg="blue")
        self.browse_button.grid(column=0, row=3, padx=10, pady=10, sticky=tk.W)

        # Indicator in the middle
        indicator_frame = tk.Frame(self.root)
        indicator_frame.grid(column=1, row=3, columnspan=2, pady=10)
        self.canvas = tk.Canvas(indicator_frame, width=100, height=100)
        self.canvas.pack()
        self.recording_circle = self.canvas.create_oval(25, 25, 75, 75, fill="Orange")
        self.recording_label = self.canvas.create_text(50, 90, text="Standby", font=("Cascadia Code", 12), fill="black")

        # Ports frame with scrollbar
        ports_frame_container = tk.Frame(self.root)
        ports_frame_container.grid(column=0, row=4, columnspan=4, pady=20, sticky=(tk.N, tk.S, tk.E, tk.W))
        ports_canvas = tk.Canvas(ports_frame_container)
        ports_scrollbar = ttk.Scrollbar(ports_frame_container, orient="vertical", command=ports_canvas.yview)
        ports_canvas.configure(yscrollcommand=ports_scrollbar.set)
        ports_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        ports_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.ports_frame = tk.Frame(ports_canvas)
        ports_canvas.create_window((0,0), window=self.ports_frame, anchor="nw")
        self.ports_frame.bind("<Configure>", lambda event: ports_canvas.configure(scrollregion=ports_canvas.bbox("all")))

        # If no devices connected at start
        if not self.serial_ports:
            tk.Label(self.ports_frame, text="Connect your FED3 devices!", font=("Cascadia Code", 14), fg="red").pack()

        # Log Section
        log_frame = tk.Frame(self.root)
        log_frame.grid(column=0, row=5, columnspan=4, pady=10, sticky=(tk.N, tk.S, tk.E, tk.W))
        self.log_text = tk.Text(log_frame, height=10, width=130, font=("Cascadia Code", 10))
        self.log_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        log_scrollbar = ttk.Scrollbar(log_frame, orient="vertical", command=self.log_text.yview)
        log_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.log_text.configure(yscrollcommand=log_scrollbar.set)

        # Bottom Frame
        bottom_frame = tk.Frame(self.root)
        bottom_frame.grid(column=0, row=6, columnspan=4, pady=10, sticky=(tk.S, tk.E, tk.W))
        tk.Label(bottom_frame, text="© 2024 McCutcheonLab | UiT | Norway", font=("Cascadia Code", 10), fg="royalblue").pack(pady=5)
        hyperlink_label = tk.Label(bottom_frame, text="Developed by Hamid Taghipourbibalan", font=("Cascadia Code", 10, "italic"), fg="blue", cursor="hand2")
        hyperlink_label.pack(pady=5)
        hyperlink_label.bind("<Button-1>", lambda e: self.open_hyperlink("https://www.linkedin.com/in/hamid-taghipourbibalan-b7239088/"))

    def open_hyperlink(self, URL):
        webbrowser.open_new(URL)

    def show_instruction_popup(self):
        messagebox.showinfo("Instructions", "Make a right poke on FED3 to find the port!")

    def browse_json(self):
        filename = filedialog.askopenfilename(title="Select JSON File", filetypes=[("JSON Files", "*.json")])
        if filename:
            self.json_path.set(filename)

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")
        if self.save_path:
            self.log_queue.put(f"Data folder selected: {self.save_path}")

    def start_identification_threads(self):
        for port in list(self.unidentified_ports):
            self.start_identification_thread(port)

    def start_identification_thread(self, port):
        if port in self.identification_threads:
            return
        stop_event = threading.Event()
        self.identification_stop_events[port] = stop_event

        def identification_with_delay():
            time.sleep(1)
            self.identification_thread(port, stop_event)

        t = threading.Thread(target=identification_with_delay)
        t.daemon = True
        t.start()
        self.identification_threads[port] = t
        self.log_queue.put(f"Started identification thread for {port}.")

    def stop_identification_threads(self):
        for port, event in list(self.identification_stop_events.items()):
            event.set()
        for port, t in list(self.identification_threads.items()):
            t.join()
            self.log_queue.put(f"Stopped identification thread for {port}.")
            del self.identification_threads[port]
            del self.identification_stop_events[port]

    def identification_thread(self, port, stop_event):
        if port in self.port_threads:
            return
        try:
            ser = serial.Serial(port, 115200, timeout=0.1)
            event_index = column_headers.index("Event") - 1
            device_number_index = column_headers.index("Device_Number") - 1
            device_number_found = None
            while not stop_event.is_set():
                try:
                    data = ser.readline().decode('utf-8', errors='replace').strip()
                    if data:
                        data_list = data.split(",")[1:]
                        if len(data_list) == len(column_headers)-1:
                            event_value = data_list[event_index].strip()
                            # Trigger RIGHT_POKE even before start
                            if event_value == "Right":
                                # Temporarily store data in a queue to blink indicator
                                # If no widget yet, we must create a temporary widget now because user sees the poke
                                # We'll create a temporary widget if not device_number known yet
                                if port not in self.port_widgets:
                                    self.create_new_widget_for_unidentified_port(port)
                                self.port_queues[port].put("RIGHT_POKE")

                            dn = data_list[device_number_index].strip()
                            if dn:
                                device_number_found = dn
                                break
                except serial.SerialException:
                    break
                except:
                    pass
            if device_number_found:
                self.log_queue.put(f"Identified device_number={device_number_found} on port={port}")
                self.register_device_number(port, device_number_found)
            else:
                # If not identified but thread ends, do nothing special
                pass
        except serial.SerialException:
            pass
        finally:
            try:
                ser.close()
            except:
                pass

    def create_new_widget_for_unidentified_port(self, port):
        # This function creates a widget box for a port that poked right before identification ended.
        # Normally we wait until identification to create the widget.
        if port in self.port_widgets:
            return
        idx = len(self.port_widgets)
        port_name = os.path.basename(port)
        frame = ttk.LabelFrame(self.ports_frame, text=f"Port {port_name}")
        frame.grid(column=idx % 2, row=idx // 2, padx=10, pady=10, sticky=tk.W)
        status_label = ttk.Label(frame, text="Not Ready", font=("Cascadia Code", 10, "italic"), foreground="red")
        status_label.grid(column=0, row=0, sticky=tk.W)
        text_widget = tk.Text(frame, width=40, height=6, font=("Cascadia Code", 9))
        text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))
        indicator_canvas = tk.Canvas(frame, width=20, height=20)
        indicator_canvas.grid(column=1, row=0, padx=5)
        indicator_circle = indicator_canvas.create_oval(5, 5, 15, 15, fill="gray")

        self.port_widgets[port] = {
            'frame': frame,
            'status_label': status_label,
            'text_widget': text_widget,
            'indicator_canvas': indicator_canvas,
            'indicator_circle': indicator_circle
        }
        self.port_queues[port] = queue.Queue()

    def register_device_number(self, port, device_number):
        # Identification done, device_number found
        # Remove from unidentified_ports
        if port in self.unidentified_ports:
            self.unidentified_ports.remove(port)

        if device_number in self.device_number_to_widgets:
            # Known device_number, reuse the old widget
            old_widgets = self.device_number_to_widgets[device_number]
            # If a different widget was created for this port, remove it
            if port in self.port_widgets and self.port_widgets[port] is not old_widgets:
                # Means a temporary widget was created
                new_widgets = self.port_widgets[port]
                if new_widgets != old_widgets: 
                    new_widgets['frame'].destroy()
                    del self.port_widgets[port]
                    del self.port_queues[port]
            # Map port to this device_number
            self.port_to_device_number[port] = device_number
            self.device_number_to_port[device_number] = port
        else:
            # New device_number: create a new widget if none exists for this port
            if port not in self.port_widgets:
                # Create now since we know it's a new device
                self.create_new_widget_for_unidentified_port(port)

            self.port_to_device_number[port] = device_number
            self.device_number_to_port[device_number] = port
            self.device_number_to_widgets[device_number] = self.port_widgets[port]

        # Try opening port to set status ready
        self.try_open_port_for_status(port)

        # If logging active, start logging for this device
        if self.logging_active:
            self.start_logging_for_port(port)

    def try_open_port_for_status(self, port):
        try:
            ser = serial.Serial(port, 115200, timeout=1)
            ser.close()
            if port in self.port_widgets:
                self.port_widgets[port]['status_label'].config(text="Ready", foreground="green")
        except serial.SerialException:
            if port in self.port_widgets:
                self.port_widgets[port]['status_label'].config(text="Not Ready", foreground="red")

    def start_logging(self):
        self.stop_identification_threads()
        self.stop_event.clear()
        self.logging_active = True

        # Normalize inputs
        self.experimenter_name.set(self.experimenter_name.get().strip().lower())
        self.experiment_name.set(self.experiment_name.get().strip().lower())
        self.json_path.set(self.json_path.get().strip())
        self.spreadsheet_id.set(self.spreadsheet_id.get().strip())

        if not self.experimenter_name.get() or not self.experiment_name.get():
            messagebox.showerror("Error", "Please provide your name and experiment name.")
            return
        if not self.json_path.get() or not self.spreadsheet_id.get() or not self.save_path:
            messagebox.showerror("Error", "Please provide the JSON file, Spreadsheet ID, and data folder.")
            return

        try:
            creds = Credentials.from_service_account_file(self.json_path.get(), scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            self.log_queue.put("Connected to Google Sheets!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to connect to Google Sheets: {e}")
            return

        self.disable_input_fields()
        self.canvas.itemconfig(self.recording_circle, fill="yellow")
        self.canvas.itemconfig(self.recording_label, text="Logging...", fill="black")

        # Start logging for already identified devices
        for port in list(self.port_to_device_number.keys()):
            self.start_logging_for_port(port)

    def disable_input_fields(self):
        self.experimenter_entry.config(state='disabled')
        self.experiment_entry.config(state='disabled')
        self.json_entry.config(state='disabled')
        self.spreadsheet_entry.config(state='disabled')
        self.browse_json_button.config(state='disabled')
        self.browse_button.config(state='disabled')
        self.start_button.config(state='disabled')

    def enable_input_fields(self):
        self.experimenter_entry.config(state='normal')
        self.experiment_entry.config(state='normal')
        self.json_entry.config(state='normal')
        self.spreadsheet_entry.config(state='normal')
        self.browse_json_button.config(state='normal')
        self.browse_button.config(state='normal')
        self.start_button.config(state='normal')

    def start_logging_for_port(self, port):
        if (port in self.port_threads) or (not self.logging_active):
            return

        def attempt_connection(retries=self.retry_attempts, delay=self.retry_delay):
            for attempt in range(retries):
                try:
                    ser = serial.Serial(port, 115200, timeout=0.1)
                    self.data_to_save[port] = self.data_to_save.get(port, [])
                    device_number = self.port_to_device_number.get(port)
                    worksheet_name = f"Device_{device_number}" if device_number else f"Port_{os.path.basename(port)}"
                    t = threading.Thread(target=self.read_from_port, args=(ser, port, worksheet_name))
                    t.daemon = True
                    t.start()
                    self.port_threads[port] = t
                    if port in self.port_widgets:
                        self.port_widgets[port]['status_label'].config(text="Ready", foreground="green")
                    self.log_queue.put(f"Started logging from {port}.")
                    return
                except serial.SerialException as e:
                    self.log_queue.put(f"Attempt {attempt+1}: Error with port {port}: {e}")
                    time.sleep(delay)
            self.log_queue.put(f"Failed to connect to port {port} after {retries} attempts.")

        threading.Thread(target=attempt_connection).start()

    def stop_logging(self):
        self.stop_event.set()
        self.logging_active = False
        self.log_queue.put("Stopping logging...")
        self.hide_recording_indicator()
        self.enable_input_fields()
        threading.Thread(target=self._join_threads_and_save).start()

    def _join_threads_and_save(self):
        for port, t in list(self.port_threads.items()):
            t.join()
            self.log_queue.put(f"Logging thread for {port} has stopped.")
            del self.port_threads[port]

        self.save_all_data()
        self.data_saved = True
        self.root.after(0, self._finalize_exit)

    def _finalize_exit(self):
        messagebox.showinfo("Data Saved", "All data has been saved locally.")
        self.root.destroy()

    def save_all_data(self):
        if not self.save_path:
            self.log_queue.put("Error: No save path specified.")
            return

        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_name = self.experimenter_name.get().lower().strip()
        experiment_name = self.experiment_name.get().lower().strip()

        experimenter_folder = os.path.join(self.save_path, experimenter_name)
        experiment_folder = os.path.join(experimenter_folder, f"{experiment_name}_{current_time}")
        os.makedirs(experiment_folder, exist_ok=True)

        for port, data_rows in self.data_to_save.items():
            if data_rows:
                port_name = os.path.basename(port)
                safe_port_name = re.sub(r'[<>:"/\\|?*]', '_', port_name)
                filename_user = os.path.join(experiment_folder, f"{safe_port_name}_{current_time}.csv")
                try:
                    with open(filename_user, mode='w', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow(column_headers)
                        writer.writerows(data_rows)
                    self.log_queue.put(f"Data saved for {port} in {filename_user}.")
                except Exception as e:
                    self.log_queue.put(f"Failed to save data for {port}: {e}")

    def update_gui(self):
        # Update port text widgets
        for port_identifier, q in list(self.port_queues.items()):
            try:
                while True:
                    message = q.get_nowait()
                    if message == "RIGHT_POKE":
                        # Blink even if not started
                        self.trigger_indicator(port_identifier)
                    else:
                        if port_identifier in self.port_widgets:
                            self.port_widgets[port_identifier]['text_widget'].insert(tk.END, message + "\n")
                            self.port_widgets[port_identifier]['text_widget'].see(tk.END)
            except queue.Empty:
                pass

        # Update log messages
        try:
            while True:
                log_message = self.log_queue.get_nowait()
                self.log_text.insert(tk.END, f"{datetime.datetime.now().strftime('%m/%d/%Y %H:%M:%S')}: {log_message}\n")
                self.log_text.see(tk.END)
        except queue.Empty:
            pass

        current_time = time.time()
        if current_time - self.last_device_check_time >= 5:
            self.check_device_connections()
            self.last_device_check_time = current_time

        self.root.after(100, self.update_gui)

    def check_device_connections(self):
        current_ports = set(self.detect_serial_ports())

        # Disconnected devices
        for port in list(self.serial_ports):
            if port not in current_ports:
                self.serial_ports.remove(port)
                self.log_queue.put(f"Device on {port} disconnected.")
                if port in self.port_threads:
                    del self.port_threads[port]
                if port in self.identification_threads:
                    self.identification_stop_events[port].set()
                    self.identification_threads[port].join()
                    del self.identification_threads[port]
                    del self.identification_stop_events[port]

                # If this was unidentified port (no widget?), just remove from unidentified
                if port in self.unidentified_ports:
                    self.unidentified_ports.remove(port)

                # If this port had a widget and device_number known, we leave widget intact (device might return)
                # Set status to Not Ready if widget exists
                if port in self.port_widgets:
                    self.port_widgets[port]['status_label'].config(text="Not Ready", foreground="red")

                # We do not remove device_number_to_widgets. If device reconnects we reuse the widget.

        # New devices
        for port in current_ports:
            if port not in self.serial_ports:
                self.serial_ports.add(port)
                self.unidentified_ports.add(port)
                # Start identification for this new port
                self.start_identification_thread(port)

    def read_from_port(self, ser, port_identifier, worksheet_name):
        cached_data = []
        send_interval = 5
        last_send_time = time.time()
        jam_event_occurred = False
        device_number = self.port_to_device_number.get(port_identifier, None)

        event_index_in_headers = column_headers.index("Event")
        event_index_in_data = event_index_in_headers - 1
        device_number_index_in_headers = column_headers.index("Device_Number")
        device_number_index_in_data = device_number_index_in_headers - 1

        sheet = None

        while not self.stop_event.is_set():
            try:
                data = ser.readline().decode('utf-8', errors='replace').strip()
            except serial.SerialException as e:
                self.log_queue.put(f"Device on {port_identifier} disconnected: {e}")
                if port_identifier in self.port_widgets:
                    self.port_widgets[port_identifier]['status_label'].config(text="Not Ready", foreground="red")
                break

            if data:
                data_list = data.split(",")[1:]
                timestamp = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S.%f")[:-3]
                if len(data_list) == len(column_headers) - 1:
                    event_value = data_list[event_index_in_data].strip()
                    current_device_number = data_list[device_number_index_in_data].strip()

                    if device_number is None and current_device_number:
                        device_number = current_device_number
                        self.port_to_device_number[port_identifier] = device_number
                        if device_number not in self.device_number_to_widgets:
                            self.device_number_to_widgets[device_number] = self.port_widgets[port_identifier]
                        self.device_number_to_port[device_number] = port_identifier
                        worksheet_name = f"Device_{device_number}"

                    row_data = [timestamp] + data_list
                    cached_data.append(row_data)
                    if port_identifier in self.port_queues:
                        self.port_queues[port_identifier].put(f"Data logged: {data_list}")
                    self.data_to_save.setdefault(port_identifier, []).append(row_data)

                    if event_value == "JAM":
                        jam_event_occurred = True
                    if event_value == "Right":
                        if port_identifier in self.port_queues:
                            self.port_queues[port_identifier].put("RIGHT_POKE")
                else:
                    self.log_queue.put(f"Warning: Data length mismatch on {port_identifier}")

            current_time = time.time()
            should_send = (current_time - last_send_time >= send_interval)
            if should_send and device_number and self.gspread_client and self.spreadsheet_id.get():
                if sheet is None:
                    try:
                        spreadsheet = self.gspread_client.open_by_key(self.spreadsheet_id.get())
                        sheet = self.get_or_create_worksheet(spreadsheet, worksheet_name)
                    except Exception as e:
                        self.log_queue.put(f"Failed to access sheet for {port_identifier}: {e}")

                if sheet and cached_data:
                    try:
                        sheet.append_rows(cached_data)
                        self.log_queue.put(f"Appended {len(cached_data)} rows from {port_identifier} to Google Sheets.")
                        cached_data.clear()
                    except Exception as e:
                        self.log_queue.put(f"Failed to send data to Google Sheets for {port_identifier}: {e}")

                if jam_event_occurred and sheet:
                    try:
                        jam_row = [''] * len(column_headers)
                        jam_row[0] = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S.%f")[:-3]
                        jam_row[event_index_in_headers] = "JAM"
                        if device_number:
                            jam_row[device_number_index_in_headers] = device_number
                        sheet.append_row(jam_row)
                        self.log_queue.put(f"JAM event logged for {port_identifier}")
                        jam_event_occurred = False
                    except Exception as e:
                        self.log_queue.put(f"Failed to send JAM event to Google Sheets for {port_identifier}: {e}")

                last_send_time = current_time

            time.sleep(0.1)

        try:
            ser.close()
        except:
            pass
        self.log_queue.put(f"Closed serial port {ser.port}")

    def get_or_create_worksheet(self, spreadsheet, title):
        try:
            return spreadsheet.worksheet(title)
        except gspread.exceptions.WorksheetNotFound:
            sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
            sheet.append_row(column_headers)
            return sheet

    def trigger_indicator(self, port_identifier):
        if port_identifier not in self.port_widgets:
            return
        indicator_canvas = self.port_widgets[port_identifier]['indicator_canvas']
        indicator_circle = self.port_widgets[port_identifier]['indicator_circle']
        def blink(times):
            if times > 0:
                current_color = indicator_canvas.itemcget(indicator_circle, 'fill')
                next_color = 'red' if current_color == 'gray' else 'gray'
                indicator_canvas.itemconfig(indicator_circle, fill=next_color)
                self.root.after(250, lambda: blink(times - 1))
            else:
                indicator_canvas.itemconfig(indicator_circle, fill='gray')
        blink(6)

    def hide_recording_indicator(self):
        self.canvas.itemconfig(self.recording_circle, fill="red")
        self.canvas.itemconfig(self.recording_label, text="OFF", fill="red")

    def on_closing(self):
        if not self.data_saved:
            self.stop_identification_threads()
            self.stop_event.set()
            self.logging_active = False

            for t in list(self.identification_threads.values()):
                t.join()

            for t in list(self.port_threads.values()):
                t.join()

            self.save_all_data()
            self.data_saved = True

        self.root.destroy()


if __name__ == "__main__":
    # Splash screen
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root, duration=7000)
    splash_root.after(7000, splash_screen.close)
    splash_root.mainloop()

    # Main GUI
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()
